In [13]:
from pathlib import Path
import pandas as pd
import traceback
import sys

# Caminhos relativos ao projeto
ROOT = Path("Dados por município 12.0")
OUTPUT_PARQUET = Path("todas_emissoes.parquet")
print("ROOT =", ROOT)

ROOT = Dados por município 12.0


In [14]:
def is_lfs_pointer(path: Path, nlines: int = 5) -> bool:
    """Verifica se o arquivo começa com o ponteiro Git LFS."""
    try:
        with path.open("r", encoding="utf-8", errors="ignore") as f:
            head = "".join([next(f) for _ in range(nlines)])
        return "git-lfs.github.com/spec/v1" in head or head.strip().startswith("version https://git-lfs.github.com/spec/v1")
    except Exception:
        return False

def read_csv_flexible(path: Path, encodings=None, seps=None, chunksize=200_000):
    """Tenta ler um CSV com encodings e separadores comuns; usa chunks para arquivos grandes."""
    if encodings is None:
        encodings = ["utf-8", "latin1", "iso-8859-1"]
    if seps is None:
        seps = [",", ";", "\t"]

    # tenta leituras de amostra para descobrir separador/encoding
    for enc in encodings:
        for sep in seps:
            try:
                # ler uma amostra pequena para validar
                pd.read_csv(path, encoding=enc, sep=sep, nrows=5)
                # se não deu exceção, procede a leitura completa (por chunks se muito grande)
                filesize_mb = path.stat().st_size / (1024 * 1024)
                if filesize_mb > 50:  # ajuste threshold se quiser
                    print(f"  Lendo por chunks: {path.name} ({filesize_mb:.1f} MB) encoding={enc} sep={repr(sep)}")
                    parts = []
                    for chunk in pd.read_csv(path, encoding=enc, sep=sep, chunksize=chunksize):
                        parts.append(chunk)
                    return pd.concat(parts, ignore_index=True)
                else:
                    print(f"  Lendo arquivo: {path.name} encoding={enc} sep={repr(sep)}")
                    return pd.read_csv(path, encoding=enc, sep=sep)
            except Exception:
                continue
    # se chegou aqui, nenhuma tentativa funcionou
    raise ValueError(f"Não foi possível ler {path} com encodings/separadores testados.")


In [15]:
if not ROOT.exists():
    raise FileNotFoundError(f"Pasta root não encontrada: {ROOT.resolve()}")

arquivos = list(ROOT.rglob("*.csv"))
print(f"Encontrei {len(arquivos)} arquivos CSV (recursivamente).")

dfs = []
arquivos_lfs = []
erros = []

for i, arq in enumerate(arquivos, 1):
    print(f"[{i}/{len(arquivos)}] Processando: {arq}")
    try:
        if is_lfs_pointer(arq):
            arquivos_lfs.append(arq)
            print("  -> PARECE ponteiro LFS (ainda não baixado). Rode `git lfs pull` se necessário.")
            continue
        df_temp = read_csv_flexible(arq)
        # opcional: adicionar coluna com origem para rastreabilidade
        df_temp["_origem_arquivo"] = str(arq.relative_to(ROOT))
        dfs.append(df_temp)
        print(f"  -> OK: {len(df_temp)} linhas lidas.")
    except Exception as e:
        tb = traceback.format_exc()
        erros.append((str(arq), str(e)))
        print(f"  -> ERRO ao ler {arq.name}: {e}")

print("\nResumo:")
print("Arquivos LFS pendentes:", len(arquivos_lfs))
print("Erros de leitura:", len(erros))
print("DataFrames lidos:", len(dfs))

# listar alguns exemplos para inspeção
if arquivos_lfs:
    print("\nExemplos de arquivos que parecem ponteiros LFS (mostrando até 10):")
    for p in arquivos_lfs[:10]:
        print(" -", p)

if erros:
    print("\nExemplos de erros (até 5):")
    for fn, msg in erros[:5]:
        print(" -", fn, ":", msg)


Encontrei 44 arquivos CSV (recursivamente).
[1/44] Processando: Dados por município 12.0\AC\gases.csv
  Lendo arquivo: gases.csv encoding=utf-8 sep=','
  -> OK: 8944 linhas lidas.
[2/44] Processando: Dados por município 12.0\AM\ar4.csv
  Lendo arquivo: ar4.csv encoding=utf-8 sep=','
  -> OK: 8944 linhas lidas.
[2/44] Processando: Dados por município 12.0\AM\ar4.csv
  Lendo arquivo: ar4.csv encoding=utf-8 sep=','
  -> OK: 29078 linhas lidas.
[3/44] Processando: Dados por município 12.0\AM\ar6.csv
  Lendo arquivo: ar6.csv encoding=utf-8 sep=','
  -> OK: 29078 linhas lidas.
[4/44] Processando: Dados por município 12.0\AP\ar2.csv
  Lendo arquivo: ar2.csv encoding=utf-8 sep=','
  -> OK: 29078 linhas lidas.
[3/44] Processando: Dados por município 12.0\AM\ar6.csv
  Lendo arquivo: ar6.csv encoding=utf-8 sep=','
  -> OK: 29078 linhas lidas.
[4/44] Processando: Dados por município 12.0\AP\ar2.csv
  Lendo arquivo: ar2.csv encoding=utf-8 sep=','
  -> OK: 8516 linhas lidas.
[5/44] Processando: Dado

C:\Users\Vitoria Oliveira\AppData\Local\Temp\ipykernel_19996\2584397836.py:28: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(path, encoding=enc, sep=sep, chunksize=chunksize):


  -> OK: 101399 linhas lidas.
[23/44] Processando: Dados por município 12.0\NA\ar5.csv
  Lendo arquivo: ar5.csv encoding=utf-8 sep=','
  -> OK: 952 linhas lidas.
[24/44] Processando: Dados por município 12.0\NA\ar6.csv
  Lendo arquivo: ar6.csv encoding=utf-8 sep=','
  -> OK: 951 linhas lidas.
[25/44] Processando: Dados por município 12.0\PA\ar5.csv
  Lendo arquivo: ar5.csv encoding=utf-8 sep=','
  -> OK: 75380 linhas lidas.
[26/44] Processando: Dados por município 12.0\PA\ar6.csv
  Lendo arquivo: ar6.csv encoding=utf-8 sep=','
  -> OK: 75380 linhas lidas.
[26/44] Processando: Dados por município 12.0\PA\ar6.csv
  Lendo arquivo: ar6.csv encoding=utf-8 sep=','
  -> OK: 75380 linhas lidas.
[27/44] Processando: Dados por município 12.0\PA\gases.csv
  Lendo arquivo: gases.csv encoding=utf-8 sep=','
  -> OK: 75380 linhas lidas.
[27/44] Processando: Dados por município 12.0\PA\gases.csv
  Lendo arquivo: gases.csv encoding=utf-8 sep=','
  -> OK: 64719 linhas lidas.
[28/44] Processando: Dados p

C:\Users\Vitoria Oliveira\AppData\Local\Temp\ipykernel_19996\2584397836.py:28: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(path, encoding=enc, sep=sep, chunksize=chunksize):


  -> OK: 275006 linhas lidas.
[38/44] Processando: Dados por município 12.0\RS\ar6.csv
  Lendo por chunks: ar6.csv (185.4 MB) encoding=utf-8 sep=','


...

  -> OK: 275006 linhas lidas.
[39/44] Processando: Dados por município 12.0\SP\ar2.csv
  Lendo por chunks: ar2.csv (265.4 MB) encoding=utf-8 sep=','


C:\Users\Vitoria Oliveira\AppData\Local\Temp\ipykernel_19996\2584397836.py:28: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(path, encoding=enc, sep=sep, chunksize=chunksize):


  -> OK: 384790 linhas lidas.
[40/44] Processando: Dados por município 12.0\SP\gases.csv
  Lendo por chunks: gases.csv (222.1 MB) encoding=utf-8 sep=','


C:\Users\Vitoria Oliveira\AppData\Local\Temp\ipykernel_19996\2584397836.py:28: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(path, encoding=enc, sep=sep, chunksize=chunksize):


  -> OK: 341661 linhas lidas.
[41/44] Processando: Dados por município 12.0\TO\ar4.csv
  Lendo por chunks: ar4.csv (50.1 MB) encoding=utf-8 sep=','
  -> OK: 79436 linhas lidas.
[42/44] Processando: Dados por município 12.0\TO\ar5.csv
  Lendo por chunks: ar5.csv (50.1 MB) encoding=utf-8 sep=','
  -> OK: 79436 linhas lidas.
[42/44] Processando: Dados por município 12.0\TO\ar5.csv
  Lendo por chunks: ar5.csv (50.1 MB) encoding=utf-8 sep=','
  -> OK: 79436 linhas lidas.
[43/44] Processando: Dados por município 12.0\TO\ar6.csv
  Lendo por chunks: ar6.csv (50.2 MB) encoding=utf-8 sep=','
  -> OK: 79436 linhas lidas.
[43/44] Processando: Dados por município 12.0\TO\ar6.csv
  Lendo por chunks: ar6.csv (50.2 MB) encoding=utf-8 sep=','
  -> OK: 79436 linhas lidas.
[44/44] Processando: Dados por município 12.0\TO\gases.csv
  Lendo arquivo: gases.csv encoding=utf-8 sep=','
  -> OK: 79436 linhas lidas.
[44/44] Processando: Dados por município 12.0\TO\gases.csv
  Lendo arquivo: gases.csv encoding=ut

In [16]:
if not dfs:
    print("Nenhum DataFrame válido foi lido. Primeiro resolva os ponteiros LFS com `git lfs pull` e rode novamente.")
else:
    df_all = pd.concat(dfs, ignore_index=True)
    print("Concatenei DataFrames. Linhas:", len(df_all), "Colunas:", df_all.shape[1])
    df_all.to_parquet(OUTPUT_PARQUET, index=False)
    print("Salvo em:", OUTPUT_PARQUET)
    display(df_all.head())
    print("\nColunas:")
    print(df_all.columns.tolist())


Concatenei DataFrames. Linhas: 3434389 Colunas: 66


OSError: [Errno 28] Error writing bytes to file. Detail: [errno 28] No space left on device

In [ ]:
# Ajuste estes nomes conforme coluna real do CSV; este é apenas um exemplo útil
cols_lower = [c.lower().strip() for c in df_all.columns]
possible_city_cols = [c for c in df_all.columns if c.lower().strip() in ("municipio","cidade","município","nome_municipio","municipio_nome")]
print("Possíveis colunas de município:", possible_city_cols)

top_by_origem = df_all["_origem_arquivo"].value_counts().head(10)
print("Top 10 arquivos por número de linhas:")
print(top_by_origem)

year_cols = [c for c in df_all.columns if c.isdigit() and 1800 <= int(c) <= 2100]
print("Colunas parecendo anos:", year_cols[:20])


Possíveis colunas de município: ['Cidade']
Top 10 arquivos por número de linhas:
_origem_arquivo
SP\ar2.csv      384790
SP\gases.csv    341661
RS\ar5.csv      275006
RS\ar6.csv      275006
PR\ar5.csv      180598
GO\ar4.csv      136852
GO\ar2.csv      136852
GO\ar5.csv      136852
MT\ar6.csv      116714
MT\ar2.csv      116714
Name: count, dtype: int64
Colunas parecendo anos: ['1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989']
